# Bài tập thực hành 1.3.3: Tìm hiểu các tính năng và cách sử dụng sản phẩm AutoViz

## 3.1 Giới thiệu

### AutoViz là một thư viện Python giúp tự động hóa việc trực quan hóa dữ liệu. Nó tạo nhiều loại biểu đồ để phân tích mối quan hệ giữa các biến, phân phối dữ liệu, và các đặc tính quan trọng khác.

## 3.2 Cài đặt

In [9]:
# Cài đặt bằng lệnh:
# pip install autoviz

## 3.3 Cách sử dụng

In [10]:
# Khai báo thư viện
import pandas as pd
import numpy as np
from autoviz.AutoViz_Class import AutoViz_Class
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime


In [11]:
# Đọc dữ liệu
print("=== ĐỌC DỮ LIỆU ===")
df = pd.read_csv('../data/marketing_campaign.csv', sep='\t')
print(f"Kích thước dữ liệu: {df.shape}")
print(f"\nCác cột: {df.columns.tolist()}")

# Khám phá dữ liệu cơ bản
print("\n=== THÔNG TIN CƠ BẢN ===")
print(df.info())

print("\n=== THỐNG KÊ MÔ TẢ ===")
print(df.describe())

print("\n=== KIỂM TRA GIÁ TRỊ NULL ===")
print(df.isnull().sum())

# Xử lý dữ liệu
print("\n=== XỬ LÝ DỮ LIỆU ===")

# 1. Xử lý giá trị null trong Income
if df['Income'].isnull().sum() > 0:
    median_income = df['Income'].median()
    df['Income'].fillna(median_income, inplace=True)
    print(f"Đã điền {df['Income'].isnull().sum()} giá trị null trong Income bằng median: {median_income}")

# 2. Chuyển đổi Dt_Customer sang datetime và tạo feature mới
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%d-%m-%Y')
current_date = df['Dt_Customer'].max()
df['Customer_Days'] = (current_date - df['Dt_Customer']).dt.days
df['Customer_Years'] = df['Customer_Days'] / 365.25

# 3. Tính tuổi từ Year_Birth
df['Age'] = 2014 - df['Year_Birth']  # Dataset từ 2014

# 4. Loại bỏ outliers trong Age (nếu có)
df = df[(df['Age'] > 18) & (df['Age'] < 100)]

# 5. Tạo các feature tổng hợp
df['Total_Purchases'] = df['NumWebPurchases'] + df['NumCatalogPurchases'] + \
                        df['NumStorePurchases'] + df['NumDealsPurchases']

df['Total_Amount_Spent'] = df['MntWines'] + df['MntFruits'] + df['MntMeatProducts'] + \
                           df['MntFishProducts'] + df['MntSweetProducts'] + df['MntGoldProds']

df['Total_Accepted_Campaigns'] = df['AcceptedCmp1'] + df['AcceptedCmp2'] + \
                                  df['AcceptedCmp3'] + df['AcceptedCmp4'] + \
                                  df['AcceptedCmp5'] + df['Response']

df['Total_Children'] = df['Kidhome'] + df['Teenhome']

# 6. Tạo segment theo chi tiêu
df['Spending_Segment'] = pd.cut(df['Total_Amount_Spent'], 
                                 bins=[0, 100, 500, 1000, float('inf')],
                                 labels=['Low', 'Medium', 'High', 'Premium'])

print(f"\nKích thước dữ liệu sau xử lý: {df.shape}")

# Lưu dữ liệu đã xử lý
df.to_csv('marketing_campaign_processed.csv', index=False)
print("\nĐã lưu dữ liệu đã xử lý vào 'marketing_campaign_processed.csv'")

# Phân tích với AutoViz
print("\n=== PHÂN TÍCH VỚI AUTOVIZ ===")
print("Đang tạo visualizations tự động...")

# Khởi tạo AutoViz
AV = AutoViz_Class()

# Chọn các cột quan trọng để visualize (loại bỏ ID và datetime)
cols_to_viz = df.select_dtypes(include=[np.number]).columns.tolist()
cols_to_viz = [col for col in cols_to_viz if col not in ['ID', 'Z_CostContact', 'Z_Revenue']]

# Tạo một DataFrame con với các cột cần visualize
df_viz = df[cols_to_viz + ['Education', 'Marital_Status', 'Spending_Segment']].copy()

# Chạy AutoViz
try:
    dft = AV.AutoViz(
        filename='',
        dfte=df_viz,
        depVar='Total_Amount_Spent',  # Biến mục tiêu
        verbose=1,
        lowess=False,
        chart_format='png',
        max_rows_analyzed=2000,
        max_cols_analyzed=30
    )
    print("\n✓ AutoViz đã hoàn thành!")
    print("Các biểu đồ đã được lưu trong thư mục 'AutoViz_Plots'")
except Exception as e:
    print(f"\nLỗi khi chạy AutoViz: {e}")
    print("Thử phân tích thủ công...")

# Phân tích bổ sung
print("\n=== PHÂN TÍCH BỔ SUNG ===")

print("\n1. Phân bố theo Education:")
print(df['Education'].value_counts())

print("\n2. Phân bố theo Marital_Status:")
print(df['Marital_Status'].value_counts())

print("\n3. Phân bố theo Spending_Segment:")
print(df['Spending_Segment'].value_counts())

print("\n4. Tỷ lệ chấp nhận chiến dịch:")
campaign_acceptance = {
    'Campaign 1': df['AcceptedCmp1'].sum(),
    'Campaign 2': df['AcceptedCmp2'].sum(),
    'Campaign 3': df['AcceptedCmp3'].sum(),
    'Campaign 4': df['AcceptedCmp4'].sum(),
    'Campaign 5': df['AcceptedCmp5'].sum(),
    'Last Campaign': df['Response'].sum()
}
for camp, count in campaign_acceptance.items():
    print(f"{camp}: {count} ({count/len(df)*100:.2f}%)")

print("\n5. Thống kê chi tiêu theo danh mục:")
spending_categories = {
    'Wines': df['MntWines'].sum(),
    'Fruits': df['MntFruits'].sum(),
    'Meat Products': df['MntMeatProducts'].sum(),
    'Fish Products': df['MntFishProducts'].sum(),
    'Sweet Products': df['MntSweetProducts'].sum(),
    'Gold Products': df['MntGoldProds'].sum()
}
for cat, amount in spending_categories.items():
    print(f"{cat}: ${amount:,.0f} ({amount/df['Total_Amount_Spent'].sum()*100:.2f}%)")

print("\n6. Correlation với Total_Amount_Spent:")
correlations = df[['Income', 'Age', 'Recency', 'Total_Children', 
                   'Total_Purchases', 'NumWebVisitsMonth']].corrwith(df['Total_Amount_Spent'])
print(correlations.sort_values(ascending=False))

# Tạo một số visualizations thủ công
print("\n=== TẠO BIỂU ĐỒ BỔ SUNG ===")

plt.style.use('seaborn-v0_8-darkgrid')
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Phân bố chi tiêu
axes[0, 0].hist(df['Total_Amount_Spent'], bins=50, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Phân bố Tổng Chi Tiêu', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Total Amount Spent')
axes[0, 0].set_ylabel('Frequency')

# 2. Chi tiêu theo Education
df.groupby('Education')['Total_Amount_Spent'].mean().sort_values().plot(
    kind='barh', ax=axes[0, 1], color='coral'
)
axes[0, 1].set_title('Chi Tiêu Trung Bình theo Trình Độ Học Vấn', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Average Spending')

# 3. Tỷ lệ chấp nhận chiến dịch
campaign_data = pd.Series(campaign_acceptance)
axes[1, 0].bar(range(len(campaign_data)), campaign_data.values, color='lightgreen', edgecolor='black')
axes[1, 0].set_xticks(range(len(campaign_data)))
axes[1, 0].set_xticklabels(campaign_data.index, rotation=45, ha='right')
axes[1, 0].set_title('Số Lượng Chấp Nhận theo Chiến Dịch', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Count')

# 4. Correlation heatmap cho các biến quan trọng
important_cols = ['Income', 'Age', 'Total_Amount_Spent', 'Total_Purchases', 
                  'Recency', 'NumWebVisitsMonth']
corr_matrix = df[important_cols].corr()
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            ax=axes[1, 1], square=True, cbar_kws={'shrink': 0.8})
axes[1, 1].set_title('Correlation Matrix', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('marketing_analysis_summary.png', dpi=300, bbox_inches='tight')
print("✓ Đã lưu biểu đồ tổng hợp vào 'marketing_analysis_summary.png'")
plt.show()

print("\n=== HOÀN TẤT ===")
print(f"Tổng số khách hàng: {len(df)}")
print(f"Tổng doanh thu: ${df['Total_Amount_Spent'].sum():,.0f}")
print(f"Chi tiêu trung bình/khách hàng: ${df['Total_Amount_Spent'].mean():,.2f}")
print(f"Tỷ lệ chấp nhận chiến dịch trung bình: {df['Total_Accepted_Campaigns'].mean():.2%}")

=== ĐỌC DỮ LIỆU ===
Kích thước dữ liệu: (2240, 29)

Các cột: ['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome', 'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response']

=== THÔNG TIN CƠ BẢN ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Ki

,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
Income,float64,0.000000,NA,1730.000000,666666.000000,Column has 6 outliers greater than upper bound (116904.00) or lower than lower bound(-12664.00). Cap them or remove them.
Customer_Years,float64,0.000000,NA,0.000000,1.913758,No issue
Year_Birth,int64,0.000000,2,1940.000000,1995.000000,Possible date-time colum: transform before modeling step.
Kidhome,int64,0.000000,0,0.000000,2.000000,No issue
Teenhome,int64,0.000000,0,0.000000,2.000000,No issue
Recency,int64,0.000000,5,0.000000,99.000000,No issue
MntWines,int64,0.000000,39,0.000000,1493.000000,Column has 29 outliers greater than upper bound (1226.50) or lower than lower bound(-697.50). Cap them or remove them.
MntFruits,int64,0.000000,8,0.000000,199.000000,Column has 189 outliers greater than upper bound (81.00) or lower than lower bound(-47.00). Cap them or remove them.
MntMeatProducts,int64,0.000000,28,0.000000,1622.000000,Column has 159 outliers greater than upper bound (541.00) or lower than lower bound(-299.00). Cap them or remove them.
MntFishProducts,int64,0.000000,9,0.000000,259.000000,Column has 182 outliers greater than upper bound (119.50) or lower than lower bound(-68.50). Cap them or remove them.


Number of All Scatter Plots = 3
Could not draw some Time Series plots. agg function failed [how->mean,dtype->object]
All Plots done
Time to run AutoViz = 5 seconds 

 ###################### AUTO VISUALIZATION Completed ########################

✓ AutoViz đã hoàn thành!
Các biểu đồ đã được lưu trong thư mục 'AutoViz_Plots'

=== PHÂN TÍCH BỔ SUNG ===

1. Phân bố theo Education:
Education
Graduation    1127
PhD            485
Master         370
2n Cycle       200
Basic           53
Name: count, dtype: int64

2. Phân bố theo Marital_Status:
Marital_Status
Married     862
Together    579
Single      479
Divorced    231
Widow        77
Alone         3
Absurd        2
YOLO          2
Name: count, dtype: int64

3. Phân bố theo Spending_Segment:
Spending_Segment
Low        711
Premium    601
Medium     531
High       392
Name: count, dtype: int64

4. Tỷ lệ chấp nhận chiến dịch:
Campaign 1: 144 (6.44%)
Campaign 2: 30 (1.34%)
Campaign 3: 162 (7.25%)
Campaign 4: 167 (7.47%)
Campaign 5: 162 (7.25%)

# Kết thúc